**This notebook contains a script that creates student submission folders for SETA**

**Add Local Library to Path**

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

**Import Libraries**

In [2]:
import pandas as pd
import re
from local_library import create_folder
from local_library import import_worksheet
from local_library import export_worksheet


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


**Import Data**

In [3]:
#import cohort data
cohort_df = import_worksheet("Students 02-08-22", "Students (3).csv")

**Update Columns Names**

In [4]:

column_names = {"First Name":"learner_name", "Last Name": "learner_surname",
                "Email Address":"learner_email", "Middle Name(s)":"middle_name"
               }

cohort_df = cohort_df.rename(columns=column_names)

**Create Fullname**

In [5]:
cohort_df['middle_name'] = cohort_df['middle_name'].fillna("")
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['middle_name'].str.strip().str.lower()
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.title()
cohort_df['learner_fullname'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['learner_surname'].str.strip().str.lower()
cohort_df['learner_fullname'] = [re.sub(' +', ' ', fullname) for fullname in cohort_df['learner_fullname']]
cohort_df['learner_fullname'] = cohort_df['learner_fullname'].str.title()


**Subset Useful Columns and Rows**

In [6]:
column_names = ['Username', 'Campus', 'learner_email', 'learner_fullname']

cohort_df = cohort_df[cohort_df['Registration Status'] == 'Active']
cohort_df = cohort_df[cohort_df['Seta Eligible'] != 'No']
cohort_df = cohort_df[column_names]

In [7]:
cohort_df['folder_id'] = ''
len(cohort_df)

339

**Create Folders**

In [8]:
for index, row in cohort_df.iterrows():
    parent_id = create_folder("1N-cPeP3oqULgjcBXtMUHwIxTwLHSOjnQ",row['learner_fullname'])
    exams_id = create_folder(parent_id, "exams")
    create_folder(exams_id, "formative")
    create_folder(exams_id, "summative")
    cohort_df.loc[cohort_df['learner_fullname'] == row['learner_fullname'], 'folder_id'] = parent_id


In [9]:
export_worksheet("POE Compilation 2021","Evidence Submission", cohort_df)

In [3]:
#cohort_df = import_worksheet("POE Compilation 2021","Evidence Submission")

In [4]:
cohort_df.columns

Index(['Username', 'learner_email', 'learner_fullname', 'folder_id'], dtype='object')